## Enrich routes and plot heatmap

### To do:

**Enrich routes**

Right now I use one only one route from station x to station y, I suspect we have different routes, because when I include the waytypes data we have more unique that actualy station x to station y. 

We could introduce randomness to our system by picking random routes between two stations. Lets say we havd 5 routes from station X to staion Y, for every rental pick a random of the 5 routes. 

Or we could assume that people only took one of these routes. Maybe the shortest? 

- [ ] Check if there are different routes in *data_all_gpd_v3* from station X to station Y.
- [ ] If there is, make a decision about randomness or not.

**Plot heatmap**

Right now we have LineStrings for each routes, meaning a combination of lines making up the whole routes. 

To check the most used routes, we could split the routes at all junctions and count for all rentals in the enriched bike_data how many times each part of the roads are used.

- [ ] Split all routes into smaller pieces
- [ ] Count how many times each smaller piece was used in a routes and store it in a dict
- [ ] Highlight which of these smaller pieces are actually bikelanes and which are not

## Imports and load data

In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import LineString
import sys
from tqdm import tqdm

/Users/thomas/.pyenv/versions/3.9.12/envs/gds/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/var/folders/18/lkk4lh9d5hb1bmvlwb1kcsth0000gn/T/ipykernel_57420/1745742774.py:1: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to S

We want to see how many unique routes there are in the dataset, as well as in part of the data we enriched with routes-data. 

In [2]:
bike_data = pd.read_csv("../data/july_2022_filtered_renamed.csv")
bike_data = bike_data.drop(columns=["Unnamed: 0"])
print(bike_data.shape)

(131704, 15)


In [3]:
bike_data.head().T

,0,1,2,3,4
started_at,2022-07-01 03:00:14.428000+00:00,2022-07-01 03:00:25.577000+00:00,2022-07-01 03:04:57.874000+00:00,2022-07-01 03:05:27.456000+00:00,2022-07-01 03:11:37.601000+00:00
ended_at,2022-07-01 03:16:28.492000+00:00,2022-07-01 03:16:23.183000+00:00,2022-07-01 03:14:42.301000+00:00,2022-07-01 03:11:22.103000+00:00,2022-07-01 03:47:45.110000+00:00
duration,974,957,584,354,2167
st_st_id,447,537,563,442,464
st_st_name,Kværnerbyen,St. Olavs gate,Huitfeldts gate,Vulkan,Sukkerbiten
st_st_desc,langs Smeltedigelen,ved Pilestredet,ved Observatoriegata,ved Maridalsveien,ved gangbroen
st_st_lat,59.904473,59.917968,59.912684,59.92251,59.905124
st_st_lon,10.786167,10.738629,10.7229,10.75101,10.753764
en_st_id,1755,431,2328,599,600
en_st_name,Aker Brygge,Chr. Michelsens Gate,The Hub,Paléhaven,Dyvekes bru


In [4]:
gdf = gpd.read_file("../data/data_all_gpd_v3.shp")
print(gdf.shape)
gdf.head(5)

(67890, 19)


,started_at,ended_at,duration,st_st_id,st_st_name,st_st_desc,st_st_lat,st_st_lon,en_st_id,en_st_name,en_st_desc,en_st_lat,en_st_lon,minutes,id,total_dist,waytypes_d,waytypes_s,geometry
0,2022-07-02 08:39:02.618000+00:00,2022-07-02 08:47:54.958000+00:00,532,408,Tøyen skole,forsiden av skolebygget,59.914943,10.773977,448,Oslo Plaza,ved rundkjøringen,59.912183,10.754434,8,6044,1502.2,0 3 7;3 13 2;13 20 3;20 50 2;50 82 3;82 91 7;,v:3.0 d:805.9 a:53.65;v:2.0 d:638.3 a:42.49;v:...,"LINESTRING (10.77395 59.91491, 10.77397 59.914..."
1,2022-07-02 08:39:04.505000+00:00,2022-07-02 08:48:36.381000+00:00,571,397,Storo Storsenter,langs Vitaminveien,59.946710,10.773805,421,Alexander Kiellands Plass,langs Maridalsveien,59.928067,10.751203,9,6045,2701.3,0 2 7;2 86 2;86 94 7;94 111 6;111 112 7;112 11...,v:2.0 d:1435.3 a:53.14;v:3.0 d:805.4 a:29.82;v...,"LINESTRING (10.77378 59.94668, 10.77369 59.946..."
2,2022-07-02 08:39:22.379000+00:00,2022-07-02 08:55:10.095000+00:00,947,601,Rolf Wickstrøms vei,ved Bergrådveien,59.953411,10.757724,2309,Ulven Torg,"innenfor byggegjerdene, sør for Ulvenveien",59.924721,10.810727,15,6047,5583.8,0 4 7;4 83 6;83 93 7;93 141 6;141 142 7;142 16...,v:6.0 d:2775.3 a:49.7;v:3.0 d:1226.1 a:21.96;v...,"LINESTRING (10.75776 59.95342, 10.75783 59.953..."
3,2022-07-02 08:40:20.905000+00:00,2022-07-02 09:03:56.044000+00:00,1415,464,Sukkerbiten,ved gangbroen,59.905124,10.753764,561,Kirkeveien,ved Gørbitz' gate,59.933480,10.726294,23,6052,4430.6,0 3 7;3 7 4;7 8 7;8 18 2;18 42 1;42 139 2;139 ...,v:2.0 d:2641.3 a:59.61;v:3.0 d:841.5 a:18.99;v...,"LINESTRING (10.75380 59.90512, 10.75386 59.905..."
4,2022-07-02 08:40:45.470000+00:00,2022-07-02 08:50:08.635000+00:00,563,519,Thomas Heftyes plass,ved Bygdøy allé,59.918578,10.703900,473,Majorstuveien,langs Maries gate,59.926697,10.716195,9,6053,1409.8,0 3 7;3 30 2;30 54 1;54 64 3;,v:1.0 d:657.1 a:46.61;v:2.0 d:427.4 a:30.32;v:...,"LINESTRING (10.70389 59.91857, 10.70347 59.918..."


In [5]:
# Get a list of unique routes (start and stop stations).

routes_list_sorted = []
routes_list = []

with tqdm(total=len(gdf)) as pbar:
    for row in gdf[['st_st_id', 'en_st_id']].iterrows():
        route = list(row[1])

        if route not in routes_list:
            routes_list.append(route)

        route_sort = route
        route_sort.sort()

        if route not in routes_list_sorted:
            routes_list_sorted.append(route)

        pbar.update(1)

print('ENRICHED DATA')
print(f'Routes when the order of start and stop is important: {len(routes_list)}')
print(f'Routes when the order of start and stop is NOT important: {len(routes_list_sorted)}')

100%|██████████| 67890/67890 [00:24<00:00, 2804.00it/s]

ENRICHED DATA
Routes when the order of start and stop is important: 42241
Routes when the order of start and stop is NOT important: 18382


In [6]:
for idxA, routeA in enumerate(gdf[['st_st_id', 'en_st_id']].iterrows()):
    routeA = list(routeA[1])

    for idxB, routeB in enumerate(gdf[['st_st_id', 'en_st_id']].iterrows()):
        routeB = list(routeB[1])

        routeB_rev = [routeB[1], routeB[0]]

        #print(routeA, routeB_rev)
        if routeA == routeB_rev:
            print(idxA, idxB, routeA, routeB)
            break
    if routeA == routeB_rev:
        break

0 8174 [408, 448] [448, 408]


In [7]:
print("Check two between the same stations from different directions.")
display(gdf[['st_st_id', 'st_st_name', 'en_st_id', 'en_st_name', 'minutes', 'id', 'total_dist']].iloc[[0, 8174]])
print('We can see there is a difference in the distance and time between the two routes.')

Check two between the same stations from different directions.


,st_st_id,st_st_name,en_st_id,en_st_name,minutes,id,total_dist
0,408,Tøyen skole,448,Oslo Plaza,8,6044,1502.2
8174,448,Oslo Plaza,408,Tøyen skole,9,5210,1510.2


We can see there is a difference in the distance and time between the two routes.


In [8]:
# Get a list of unique routes (start and stop stations).

bd_routes_list_sorted = []
bd_routes_list = []

with tqdm(total=len(bike_data)) as pbar:
    for row in bike_data[['st_st_id', 'en_st_id']].iterrows():
        route = list(row[1])

        if route not in bd_routes_list:
            bd_routes_list.append(route)

        route_sort = route
        route_sort.sort()

        if route not in bd_routes_list_sorted:
            bd_routes_list_sorted.append(route)

        pbar.update(1)

print(f'Routes when the order of start and stop is important: {len(bd_routes_list)}')
print(f'Routes when the order of start and stop is NOT important: {len(bd_routes_list_sorted)}')

100%|██████████| 131704/131704 [01:14<00:00, 1757.33it/s]

Routes when the order of start and stop is important: 74644
Routes when the order of start and stop is NOT important: 23349


In [9]:
print("Number of unique routes in rental:", gdf.shape)
print("Number of unique routes in rental:", bike_data.shape)

Number of unique routes in rental: (67890, 19)
Number of unique routes in rental: (131704, 15)


In [10]:
bike_data.head(5)

,started_at,ended_at,duration,st_st_id,st_st_name,st_st_desc,st_st_lat,st_st_lon,en_st_id,en_st_name,en_st_desc,en_st_lat,en_st_lon,minutes,id
0,2022-07-01 03:00:14.428000+00:00,2022-07-01 03:16:28.492000+00:00,974,447,Kværnerbyen,langs Smeltedigelen,59.904473,10.786167,1755,Aker Brygge,ved trikkestopp,59.911184,10.730035,16,0
1,2022-07-01 03:00:25.577000+00:00,2022-07-01 03:16:23.183000+00:00,957,537,St. Olavs gate,ved Pilestredet,59.917968,10.738629,431,Chr. Michelsens Gate,ved Fjellgata,59.927292,10.773185,15,1
2,2022-07-01 03:04:57.874000+00:00,2022-07-01 03:14:42.301000+00:00,584,563,Huitfeldts gate,ved Observatoriegata,59.912684,10.722900,2328,The Hub,Mellom Oslo City og The Hub hotel,59.912522,10.750910,9,6
3,2022-07-01 03:05:27.456000+00:00,2022-07-01 03:11:22.103000+00:00,354,442,Vulkan,ved Maridalsveien,59.922510,10.751010,599,Paléhaven,langs Strandgata,59.910331,10.750110,5,7
4,2022-07-01 03:11:37.601000+00:00,2022-07-01 03:47:45.110000+00:00,2167,464,Sukkerbiten,ved gangbroen,59.905124,10.753764,600,Dyvekes bru,ved skatepark,59.905323,10.768958,36,8


In [11]:
gdf.head(5)

,started_at,ended_at,duration,st_st_id,st_st_name,st_st_desc,st_st_lat,st_st_lon,en_st_id,en_st_name,en_st_desc,en_st_lat,en_st_lon,minutes,id,total_dist,waytypes_d,waytypes_s,geometry
0,2022-07-02 08:39:02.618000+00:00,2022-07-02 08:47:54.958000+00:00,532,408,Tøyen skole,forsiden av skolebygget,59.914943,10.773977,448,Oslo Plaza,ved rundkjøringen,59.912183,10.754434,8,6044,1502.2,0 3 7;3 13 2;13 20 3;20 50 2;50 82 3;82 91 7;,v:3.0 d:805.9 a:53.65;v:2.0 d:638.3 a:42.49;v:...,"LINESTRING (10.77395 59.91491, 10.77397 59.914..."
1,2022-07-02 08:39:04.505000+00:00,2022-07-02 08:48:36.381000+00:00,571,397,Storo Storsenter,langs Vitaminveien,59.946710,10.773805,421,Alexander Kiellands Plass,langs Maridalsveien,59.928067,10.751203,9,6045,2701.3,0 2 7;2 86 2;86 94 7;94 111 6;111 112 7;112 11...,v:2.0 d:1435.3 a:53.14;v:3.0 d:805.4 a:29.82;v...,"LINESTRING (10.77378 59.94668, 10.77369 59.946..."
2,2022-07-02 08:39:22.379000+00:00,2022-07-02 08:55:10.095000+00:00,947,601,Rolf Wickstrøms vei,ved Bergrådveien,59.953411,10.757724,2309,Ulven Torg,"innenfor byggegjerdene, sør for Ulvenveien",59.924721,10.810727,15,6047,5583.8,0 4 7;4 83 6;83 93 7;93 141 6;141 142 7;142 16...,v:6.0 d:2775.3 a:49.7;v:3.0 d:1226.1 a:21.96;v...,"LINESTRING (10.75776 59.95342, 10.75783 59.953..."
3,2022-07-02 08:40:20.905000+00:00,2022-07-02 09:03:56.044000+00:00,1415,464,Sukkerbiten,ved gangbroen,59.905124,10.753764,561,Kirkeveien,ved Gørbitz' gate,59.933480,10.726294,23,6052,4430.6,0 3 7;3 7 4;7 8 7;8 18 2;18 42 1;42 139 2;139 ...,v:2.0 d:2641.3 a:59.61;v:3.0 d:841.5 a:18.99;v...,"LINESTRING (10.75380 59.90512, 10.75386 59.905..."
4,2022-07-02 08:40:45.470000+00:00,2022-07-02 08:50:08.635000+00:00,563,519,Thomas Heftyes plass,ved Bygdøy allé,59.918578,10.703900,473,Majorstuveien,langs Maries gate,59.926697,10.716195,9,6053,1409.8,0 3 7;3 30 2;30 54 1;54 64 3;,v:1.0 d:657.1 a:46.61;v:2.0 d:427.4 a:30.32;v:...,"LINESTRING (10.70389 59.91857, 10.70347 59.918..."


### Check if there are different routes for same start and end stations

In [12]:
# Check for different routes between the same stations.

for idxA, routeA in enumerate(bike_data[['st_st_id', 'en_st_id']].iterrows()):
    routeA = list(routeA[1])

    for idxB, routeB in enumerate(bike_data[['st_st_id', 'en_st_id']].iterrows()):
        routeB = list(routeB[1])

        routeB_rev = [routeB[1], routeB[0]]

        #print(routeA, routeB_rev)
        if routeA == routeB_rev:
            print(idxA, idxB, routeA, routeB)
            break
    if routeA == routeB_rev:
        break

0 5874 [447, 1755] [1755, 447]


In [13]:
gdf.head(5)

,started_at,ended_at,duration,st_st_id,st_st_name,st_st_desc,st_st_lat,st_st_lon,en_st_id,en_st_name,en_st_desc,en_st_lat,en_st_lon,minutes,id,total_dist,waytypes_d,waytypes_s,geometry
0,2022-07-02 08:39:02.618000+00:00,2022-07-02 08:47:54.958000+00:00,532,408,Tøyen skole,forsiden av skolebygget,59.914943,10.773977,448,Oslo Plaza,ved rundkjøringen,59.912183,10.754434,8,6044,1502.2,0 3 7;3 13 2;13 20 3;20 50 2;50 82 3;82 91 7;,v:3.0 d:805.9 a:53.65;v:2.0 d:638.3 a:42.49;v:...,"LINESTRING (10.77395 59.91491, 10.77397 59.914..."
1,2022-07-02 08:39:04.505000+00:00,2022-07-02 08:48:36.381000+00:00,571,397,Storo Storsenter,langs Vitaminveien,59.946710,10.773805,421,Alexander Kiellands Plass,langs Maridalsveien,59.928067,10.751203,9,6045,2701.3,0 2 7;2 86 2;86 94 7;94 111 6;111 112 7;112 11...,v:2.0 d:1435.3 a:53.14;v:3.0 d:805.4 a:29.82;v...,"LINESTRING (10.77378 59.94668, 10.77369 59.946..."
2,2022-07-02 08:39:22.379000+00:00,2022-07-02 08:55:10.095000+00:00,947,601,Rolf Wickstrøms vei,ved Bergrådveien,59.953411,10.757724,2309,Ulven Torg,"innenfor byggegjerdene, sør for Ulvenveien",59.924721,10.810727,15,6047,5583.8,0 4 7;4 83 6;83 93 7;93 141 6;141 142 7;142 16...,v:6.0 d:2775.3 a:49.7;v:3.0 d:1226.1 a:21.96;v...,"LINESTRING (10.75776 59.95342, 10.75783 59.953..."
3,2022-07-02 08:40:20.905000+00:00,2022-07-02 09:03:56.044000+00:00,1415,464,Sukkerbiten,ved gangbroen,59.905124,10.753764,561,Kirkeveien,ved Gørbitz' gate,59.933480,10.726294,23,6052,4430.6,0 3 7;3 7 4;7 8 7;8 18 2;18 42 1;42 139 2;139 ...,v:2.0 d:2641.3 a:59.61;v:3.0 d:841.5 a:18.99;v...,"LINESTRING (10.75380 59.90512, 10.75386 59.905..."
4,2022-07-02 08:40:45.470000+00:00,2022-07-02 08:50:08.635000+00:00,563,519,Thomas Heftyes plass,ved Bygdøy allé,59.918578,10.703900,473,Majorstuveien,langs Maries gate,59.926697,10.716195,9,6053,1409.8,0 3 7;3 30 2;30 54 1;54 64 3;,v:1.0 d:657.1 a:46.61;v:2.0 d:427.4 a:30.32;v:...,"LINESTRING (10.70389 59.91857, 10.70347 59.918..."


In [14]:
gdf_duplicates = gdf[['st_st_id', 'en_st_id', 'total_dist', 'waytypes_d', 'waytypes_s', 'geometry']].copy()

# remove duplicates
# gdf_duplicates.drop_duplicates(subset=['st_st_id', 'en_st_id', 'total_dist', 'waytypes_d', 'waytypes_s'], keep=False, inplace=True)

# sort values
gdf_duplicates.sort_values(by=['st_st_id', 'en_st_id', 'total_dist', 'waytypes_d', 'waytypes_s'], inplace=True)

# keep only rows where st_st_id and en_st_id are the same, but the total_dist is different or waytypes_s or way_types_s or geometry is different
gdf_duplicates = gdf_duplicates[gdf_duplicates.duplicated(keep=False)]

In [15]:
# Group the rows by the shared values in st_st_id and en_en_id columns
grouped = gdf_duplicates.groupby(['st_st_id', 'en_st_id'])

# Count the number of unique values in each group for all columns
unique_counts = grouped.nunique()

# Display the unique_counts dataframe
print(unique_counts)

# Show only the rows where the total_dist, waytypes_d, waytypes_s or geometry columns have more than one unique value
unique_counts = unique_counts[(unique_counts['total_dist'] > 1) | (unique_counts['waytypes_d'] > 1) | (unique_counts['waytypes_s'] > 1) | (unique_counts['geometry'] > 1)]

print(unique_counts)

                   total_dist  waytypes_d  waytypes_s  geometry
st_st_id en_st_id                                              
377      396                1           1           1         1
         398                1           1           1         1
         406                1           1           1         1
         421                1           1           1         1
         425                1           1           1         1
...                       ...         ...         ...       ...
2347     746                1           1           1         1
         1023               1           1           1         1
         1101               1           1           1         1
         1755               1           1           1         1
         2333               1           1           1         1

[14107 rows x 4 columns]
Empty DataFrame
Columns: [total_dist, waytypes_d, waytypes_s, geometry]
Index: []


## Enrich the bike data we didnt have time to get

We tried to download all routes by going through all rentals taking place. However if we just looked at unique routes by start and stop points, we could have reduced the required data needed to be downloaded.

Now we want to find all unique routes in the downloaded data and add it to the list of unique routes in bike_data. 

In [16]:
routes_gdf = gdf[['st_st_id', 'en_st_id', 'total_dist', 'waytypes_d', 'waytypes_s', 'geometry']].copy()

# As we used the API at different times to get route data, it probably suggested different routes.
# Therefore we have more than one route between two stations. 
# Our assumption is that people took the same route between two stations, therefore we will drop duplicates.
routes_gdf.drop_duplicates(subset = ['st_st_id', 'en_st_id'], keep = 'first', inplace = True)

# Reset the index.
routes_gdf.reset_index(drop=True, inplace=True)

# Let see the result.
routes_gdf

,st_st_id,en_st_id,total_dist,waytypes_d,waytypes_s,geometry
0,408,448,1502.2,0 3 7;3 13 2;13 20 3;20 50 2;50 82 3;82 91 7;,v:3.0 d:805.9 a:53.65;v:2.0 d:638.3 a:42.49;v:...,"LINESTRING (10.77395 59.91491, 10.77397 59.914..."
1,397,421,2701.3,0 2 7;2 86 2;86 94 7;94 111 6;111 112 7;112 11...,v:2.0 d:1435.3 a:53.14;v:3.0 d:805.4 a:29.82;v...,"LINESTRING (10.77378 59.94668, 10.77369 59.946..."
2,601,2309,5583.8,0 4 7;4 83 6;83 93 7;93 141 6;141 142 7;142 16...,v:6.0 d:2775.3 a:49.7;v:3.0 d:1226.1 a:21.96;v...,"LINESTRING (10.75776 59.95342, 10.75783 59.953..."
3,464,561,4430.6,0 3 7;3 7 4;7 8 7;8 18 2;18 42 1;42 139 2;139 ...,v:2.0 d:2641.3 a:59.61;v:3.0 d:841.5 a:18.99;v...,"LINESTRING (10.75380 59.90512, 10.75386 59.905..."
4,519,473,1409.8,0 3 7;3 30 2;30 54 1;54 64 3;,v:1.0 d:657.1 a:46.61;v:2.0 d:427.4 a:30.32;v:...,"LINESTRING (10.70389 59.91857, 10.70347 59.918..."
...,...,...,...,...,...,...
25706,462,465,3485.6,0 6 7;6 9 2;9 20 3;20 44 2;44 51 3;51 88 2;88 ...,v:2.0 d:1967.2 a:56.44;v:3.0 d:827.7 a:23.74;v...,"LINESTRING (10.73100 59.92712, 10.73149 59.927..."
25707,575,739,4120.6,0 2 7;2 63 2;63 69 6;69 186 2;186 202 3;202 20...,v:2.0 d:3743.7 a:90.85;v:3.0 d:301.3 a:7.31;v:...,"LINESTRING (10.73262 59.94744, 10.73257 59.947..."
25708,624,388,3598.0,0 2 7;2 5 6;5 162 2;162 175 1;175 192 2;192 20...,v:2.0 d:3190.8 a:88.68;v:3.0 d:318.5 a:8.85;v:...,"LINESTRING (10.76749 59.92877, 10.76752 59.928..."
25709,596,558,2867.9,0 1 7;1 2 2;2 4 6;4 24 3;24 58 2;58 86 1;86 13...,v:2.0 d:1469.3 a:51.23;v:3.0 d:898.1 a:31.32;v...,"LINESTRING (10.73603 59.93103, 10.73594 59.931..."


In [22]:
# You want to export new data?
export_new_data = False

# Enrich the bike data with the route data.
bike_data2 = routes_gdf.merge(bike_data, on=['st_st_id', 'en_st_id'], how='left')

# Drop the rows we dont have data for.
bike_data2.dropna(subset=['waytypes_d'], inplace=True)

# Change order of columns as original gdf.
bike_data2 = bike_data2[gdf.columns]

# Exprt as shp.
if export_new_data:
    bike_data2.to_file('../data/data_all_gpd_v4.shp')

# Lets see how many rows we have.
bike_data2.T

,0,1,2,3,4,5,6,7,8,9,...,114915,114916,114917,114918,114919,114920,114921,114922,114923,114924
started_at,2022-07-01 21:18:29.678000+00:00,2022-07-02 08:39:02.618000+00:00,2022-07-06 15:36:16.304000+00:00,2022-07-02 08:39:04.505000+00:00,2022-07-02 12:38:43.229000+00:00,2022-07-02 12:38:50.365000+00:00,2022-07-04 15:37:11.755000+00:00,2022-07-05 12:11:51.866000+00:00,2022-07-05 12:45:03.471000+00:00,2022-07-05 14:30:28.208000+00:00,...,2022-07-27 14:57:31.546000+00:00,2022-07-29 06:04:41.054000+00:00,2022-07-05 06:08:54.450000+00:00,2022-07-05 06:09:11.943000+00:00,2022-07-27 04:50:19.874000+00:00,2022-07-05 06:09:41.903000+00:00,2022-07-26 14:01:40.887000+00:00,2022-07-05 06:10:07.914000+00:00,2022-07-05 06:10:56.512000+00:00,2022-07-05 06:11:38.707000+00:00
ended_at,2022-07-01 21:26:29.560000+00:00,2022-07-02 08:47:54.958000+00:00,2022-07-06 15:41:54.552000+00:00,2022-07-02 08:48:36.381000+00:00,2022-07-02 12:49:29.189000+00:00,2022-07-02 12:49:26.801000+00:00,2022-07-04 15:45:49.513000+00:00,2022-07-05 12:21:46.795000+00:00,2022-07-05 12:55:33.265000+00:00,2022-07-05 14:40:20.297000+00:00,...,2022-07-27 15:18:56.422000+00:00,2022-07-29 06:14:30.324000+00:00,2022-07-05 06:14:15.078000+00:00,2022-07-05 06:19:21.820000+00:00,2022-07-27 05:00:56.332000+00:00,2022-07-05 06:25:30.553000+00:00,2022-07-26 14:18:53.785000+00:00,2022-07-05 06:23:48.691000+00:00,2022-07-05 06:18:35.020000+00:00,2022-07-05 06:20:05.391000+00:00
duration,479,532,338,571,645,636,517,594,629,592,...,1284,589,320,609,636,948,1032,820,458,506
st_st_id,408,408,408,397,397,397,397,397,397,397,...,423,423,622,462,462,575,575,624,596,403
st_st_name,Tøyen skole,Tøyen skole,Tøyen skole,Storo Storsenter,Storo Storsenter,Storo Storsenter,Storo Storsenter,Storo Storsenter,Storo Storsenter,Storo Storsenter,...,Schous plass,Schous plass,Pilestredet 63,Stensparken,Stensparken,Sognsveien,Sognsveien,Dælenenggata,Idioten,Parkveien
st_st_desc,forsiden av skolebygget,forsiden av skolebygget,forsiden av skolebygget,langs Vitaminveien,langs Vitaminveien,langs Vitaminveien,langs Vitaminveien,langs Vitaminveien,langs Vitaminveien,langs Vitaminveien,...,nærmest rundkjøringen,nærmest rundkjøringen,ved trikkestoppet,ved Sporveisgata,ved Sporveisgata,ved Ullevål Stadion T-bane,ved Ullevål Stadion T-bane,ved Fagerheimgata,langs Ullevålsveien,ved trikkestoppet
st_st_lat,59.914943,59.914943,59.914943,59.94671,59.94671,59.94671,59.94671,59.94671,59.94671,59.94671,...,59.920335,59.920335,59.923883,59.92714,59.92714,59.947433,59.947433,59.92875,59.93102,59.921768
st_st_lon,10.773977,10.773977,10.773977,10.773805,10.773805,10.773805,10.773805,10.773805,10.773805,10.773805,...,10.760804,10.760804,10.731363,10.730981,10.730981,10.73257,10.73257,10.767546,10.736037,10.730476
en_st_id,448,448,448,421,421,421,421,421,421,421,...,441,441,2280,465,465,739,739,388,558,547
en_st_name,Oslo Plaza,Oslo Plaza,Oslo Plaza,Alexander Kiellands Plass,Alexander Kiellands Plass,Alexander Kiellands Plass,Alexander Kiellands Plass,Alexander Kiellands Plass,Alexander Kiellands Plass,Alexander Kiellands Plass,...,Vippetangen øst,Vippetangen øst,Klingenberggata,Bjørvika,Bjørvika,Hammersborg Torg,Hammersborg Torg,Skovveien,Dokkveien,Skippergata


Lets see the finale results.

In [20]:
print("Shape of the original data:", gdf.shape)
print("Shape of the original data:", bike_data.shape)
print(f"\nWe got {bike_data2.shape[0]-gdf.shape[0]} more routes without the API!")

Shape of the original data: (67890, 19)
Shape of the original data: (131704, 15)

We got 47035 more routes without the API!
